In [10]:
!git clone https://github.com/terminalai/webdev-ai
!cp webdev-ai/data/* .
!rm -rf webdev-ai

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import clear_output
import re

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

Cloning into 'webdev-ai'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 16 (delta 3), reused 12 (delta 3), pack-reused 0
Unpacking objects: 100% (16/16), done.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
df = pd.read_csv("mbti.csv")
df

,text,I/E,N/S,T/F,P/J
0,yknow point id call impossible really mature w...,1,0,1,1
1,interesting person gaze penetrating speech mea...,1,0,1,1
2,pagsubok lang yan kahit gaano kahirap ang isan...,1,0,1,1
3,doesnt matter im hiding body actually help poi...,1,0,1,1
4,tell calm fuck assure meant sign damn papers w...,1,0,1,1
...,...,...,...,...,...
31995,dont worst enemy per conflicts,0,1,0,0
31996,yep im good english including reading writinge...,0,1,0,0
31997,dont know appealing building part looks fun th...,0,1,0,0
31998,struggle often notice kill conversations sayin...,0,1,0,0


In [12]:
x = df.text
y = df[["I/E", "N/S", "T/F", "P/J"]]

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.4)
x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size=0.25)

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=2000, oov_token="<OOV>")
tokenizer.fit_on_texts(x_train)
word_index = tokenizer.word_index

x_train = pad_sequences(tokenizer.texts_to_sequences(x_train), maxlen=100, padding='post', truncating='post')
x_test = pad_sequences(tokenizer.texts_to_sequences(x_test), maxlen=100, padding='post', truncating='post')
x_val = pad_sequences(tokenizer.texts_to_sequences(x_val), maxlen=100, padding='post', truncating='post')

AttributeError: ignored

In [13]:
clf = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 64), # embedding layer
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, dropout=0.2, recurrent_dropout=0.2)), # LSTM layer
    tf.keras.layers.Dropout(rate=0.2), # dropout layer
    tf.keras.layers.Dense(64, activation='relu'), # fully connected layer
    tf.keras.layers.Dense(4, activation='sigmoid') # final layer
])

clf.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'AUC'])

clf.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          128000    
                                                                 
 bidirectional (Bidirectiona  (None, 128)              66048     
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 4)                 260       
                                                                 
Total params: 202,564
Trainable params: 202,564
Non-trainable params: 0
__________________________________________________

In [14]:
early_stopping_monitor = EarlyStopping(patience=2)
history = clf.fit(x_train, y_train, epochs=10, validation_data=(x_val, y_val), callbacks = [early_stopping_monitor])

ValueError: ignored